In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [2]:
script_dir = os.getcwd()

In [19]:
logs = os.path.join(script_dir, 'combined_logs_with_age.csv')
logs_df = pd.read_csv(logs)
#display(logs_df)

,sub,trial,partner,trait,ran,order,response,age
0,103,1,3,0,1,0,3,NaN
1,103,2,2,0,1,1,1,NaN
2,103,3,1,0,1,2,1,NaN
3,103,4,3,1,1,3,1,NaN
4,103,5,2,1,1,4,0,NaN
...,...,...,...,...,...,...,...,...
1021,11072,2,2,0,1,1,0,25.0
1022,11072,3,1,0,1,2,0,25.0
1023,11072,4,3,1,1,3,0,25.0
1024,11072,5,2,1,1,4,0,25.0


In [4]:
fmri_metrics = os.path.join(script_dir, 'exclusions_tsnr-fdmean.csv') # This df stores tsnr and fdmean 
metrics_df = pd.read_csv(fmri_metrics)
#display(metrics_df)

,Subject,Run,vsmean_stan,fd_mean
0,104,1,91.703647,0.081289
1,104,2,94.158314,0.078489
2,105,1,107.323792,0.084580
3,105,2,106.397772,0.078785
4,106,1,106.132565,0.161061
...,...,...,...,...
279,10983,2,57.124313,0.262444
280,10984,1,40.239686,0.232690
281,10984,2,38.185517,0.181121
282,11031,1,56.074851,0.129847


In [5]:
fsl_inputs = os.path.join(script_dir, 'L3inputs.csv') # This df stores the relevant FSL inputs
fsl_df = pd.read_csv(fsl_inputs)
#print(fsl_df)

                                                  path
0    /ZPOOL/data/projects/sharedreward-aging/deriva...
1    /ZPOOL/data/projects/sharedreward-aging/deriva...
2    /ZPOOL/data/projects/sharedreward-aging/deriva...
3    /ZPOOL/data/projects/sharedreward-aging/deriva...
4    /ZPOOL/data/projects/sharedreward-aging/deriva...
..                                                 ...
137  /ZPOOL/data/projects/sharedreward-aging/deriva...
138  /ZPOOL/data/projects/sharedreward-aging/deriva...
139  /ZPOOL/data/projects/sharedreward-aging/deriva...
140  /ZPOOL/data/projects/sharedreward-aging/deriva...
141  /ZPOOL/data/projects/sharedreward-aging/deriva...

[142 rows x 1 columns]


In [6]:
#flip_angle =  os.path.join(script_dir, 'L3inputs.csv') # This df stores the flip angle designation to be used for dummy variable coding
#angle_df = pd.read_csv(flip_angle)

In [7]:
# This cell is used to clean the logs files so that each sub only has one row

# Pivot the dataframe to reshape it so that each subject has a single row
pivot_df = logs_df.pivot_table(index='sub', columns=['partner', 'trait'], values='response')

# Flatten the MultiIndex columns
pivot_df.columns = [f'partner_{col[0]}_trait_{col[1]}' for col in pivot_df.columns]

# Merge the age information back
pivot_df = pivot_df.merge(logs_df[['sub', 'age']].drop_duplicates(), on='sub', how='left')

# Reset index to make 'sub' a column again
pivot_df.reset_index(inplace=True)

# Print the shape of the reformatted dataframe
#print("Shape of the reformatted dataframe:", pivot_df.shape)
#display(pivot_df.head())

Shape of the reformatted dataframe: (166, 9)


,index,sub,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,age
0,0,103,1.0,0.0,1.0,0.0,3.0,1.0,NaN
1,1,104,5.0,-1.0,5.0,0.0,5.0,0.0,20.0
2,2,107,0.0,0.0,5.0,0.0,5.0,-2.0,23.0
3,3,108,1.0,0.0,4.0,-3.0,5.0,-5.0,21.0
4,4,111,5.0,0.0,5.0,5.0,5.0,5.0,80.0


In [8]:
# This cell is responsbile for adding the correct L3 input to the subject row
pd.set_option('display.max_colwidth', None)
fsl_df['sub'] = fsl_df['path'].str.extract(r'sub-(\d+)').astype(int)
fsl_df['run'] = fsl_df['path'].str.extract(r'run-(\d+)').fillna('3').astype(int)
# display(fsl_df)

In [17]:
# Now that we have our list of inputs, we can pull out the proper tsnr and fdmean for each subject
# Metric values should be derived from the same L1 run as the L3 input, if L2, use the average of both runs
# We are also going to do the math in this cell for averaging the variables

# Pivot the DataFrame to reshape it
parsed_metrics_df = metrics_df.pivot(index='Subject', columns='Run')

# Flatten the MultiIndex columns
parsed_metrics_df.columns = [f'{var}_{run}' for var, run in parsed_metrics_df.columns]

# Create the averages and the columns for them
parsed_metrics_df['avg_tsnr'] = parsed_metrics_df[['vsmean_stan_1', 'vsmean_stan_2']].mean(axis=1)
parsed_metrics_df['avg_fdmean'] = parsed_metrics_df[['fd_mean_1', 'fd_mean_2']].mean(axis=1)

# Print the reshaped DataFrame
#display(parsed_metrics_df)

,vsmean_stan_1,vsmean_stan_2,fd_mean_1,fd_mean_2,avg_tsnr,avg_fdmean
Subject,,,,,,
104,91.703647,94.158314,0.081289,0.078489,92.930981,0.079889
105,107.323792,106.397772,0.084580,0.078785,106.860782,0.081682
106,106.132565,95.061572,0.161061,0.165864,100.597069,0.163462
107,106.384215,108.987225,0.078820,0.087712,107.685720,0.083266
108,80.922331,77.318246,0.123056,0.163046,79.120289,0.143051
...,...,...,...,...,...,...
10974,NaN,47.765127,NaN,0.292531,47.765127,0.292531
10977,43.972848,46.337484,0.089736,0.094241,45.155166,0.091988
10983,55.970331,57.124313,0.296016,0.262444,56.547322,0.279230


In [10]:
# Load the reformatted CSV file
data = pivot_df
# List of partner combinations
partner_pairs = [(3, 2), (3, 1)]

# Create a new DataFrame to store the differences
diff_df = pd.DataFrame()

# Iterate over each trait
for trait in [0, 1]:
    for (p1, p2) in partner_pairs:
        # Generate the column names
        col1 = f'partner_{p1}_trait_{trait}'
        col2 = f'partner_{p2}_trait_{trait}'
        
        # Calculate the difference
        diff_col_name = f'diff_partner_{p1}_{p2}_trait_{trait}'
        diff_df[diff_col_name] = data[col1] - data[col2]

# Merge the differences with the original data
result_df = pd.concat([data, diff_df], axis=1)

# Print the first few rows of the resulting DataFrame
print("Resulting DataFrame with Differences:")
display(result_df.head())

Resulting DataFrame with Differences:


,index,sub,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,age,diff_partner_3_2_trait_0,diff_partner_3_1_trait_0,diff_partner_3_2_trait_1,diff_partner_3_1_trait_1
0,0,103,1.0,0.0,1.0,0.0,3.0,1.0,NaN,2.0,2.0,1.0,1.0
1,1,104,5.0,-1.0,5.0,0.0,5.0,0.0,20.0,0.0,0.0,0.0,1.0
2,2,107,0.0,0.0,5.0,0.0,5.0,-2.0,23.0,0.0,5.0,-2.0,-2.0
3,3,108,1.0,0.0,4.0,-3.0,5.0,-5.0,21.0,1.0,4.0,-2.0,-5.0
4,4,111,5.0,0.0,5.0,5.0,5.0,5.0,80.0,0.0,0.0,0.0,5.0


In [11]:
# Merge vsmean_stan and fd_mean columns based on the 'sub' column and display the table
result_df = pd.merge(result_df, metrics_df[['sub', 'average_vs_mean', 'average_fd_mean']], on='sub', how='left')
display(result_df)

KeyError: "None of [Index(['sub', 'average_vs_mean', 'average_fd_mean'], dtype='object')] are in the [columns]"

In [ ]:
# Merge L3 inputs based on sub-### input
result_df = pd.merge(result_df, metrics_df[['sub', 

In [ ]:
# Exclude the subject ID column
data = result_df
table = result_df[['age', 'diff_partner_3_2_trait_0', 'diff_partner_3_2_trait_1', 'diff_partner_3_1_trait_0', 'diff_partner_3_1_trait_1', 'average_fd_mean', 'average_vs_mean']]

print(table.corr)

In [ ]:
# Custom labels
custom_labels = {
    'diff_partner_3_2_trait_0': 'F-S (win)',
    'diff_partner_3_2_trait_1': 'F-S (loss)',
    'diff_partner_3_1_trait_0': 'F-C (win)',
    'diff_partner_3_1_trait_1': 'F-C (loss)'
}

# Apply custom labels
custom_labels_list = [custom_labels.get(item, item) for item in table.columns]

# Set the figure size
plt.figure(figsize=(12, 8))

# Generate the heatmap
heatmap = sns.heatmap(table.corr(), cmap='vlag', annot=True, fmt=".2f", linewidths=.5)

# Apply custom labels
heatmap.set_xticklabels(custom_labels_list, rotation=45, ha='right')
heatmap.set_yticklabels(custom_labels_list, rotation=0)

# Display the heatmap
plt.show()